# 🌋 [PASS Summit](https://pass.org/summit)

## 🐋 Databases

In [0]:
if db_id('FleitasArts') is null create database FleitasArts;
go
if db_id('tpcxbb_1gb') is null
begin
    create database tpcxbb_1gb 
    on  (filename = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\tpcxbb_1gb.mdf'),
        (filename = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\tpcxbb_1gb.ldf')   
    for attach;
end
go 

## 🐋 Config & Grant Access (Restart SQLSvc)

In [0]:
exec sp_configure 'external scripts enabled', 1;
reconfigure with override;
go 
declare @sql nvarchar(max);
select @sql = N'if not exists (select 1 from syslogins where name ='''+ @@servername +'\SQLRUserGroup'') 
begin create login ['+@@servername+'\SQLRUserGroup] from windows end';
exec sp_executesql @sql;
go 

## 🚧 Install MLM (ps as admin)

In [6]:
!powershell.exe ".\Install-MLModels.ps1 MSSQLSERVER"

## 🚧 Add To SQL Server Pre-Trained MLM (cmd as admin)

In [0]:
%%cmd
cd C:\Program Files\Microsoft SQL Server\150\Setup Bootstrap\SQL2019RC1\x64\
RSetup.exe /install /component MLM /version 9.4.7.0 /language 1033 /destdir "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\Lib\site-packages\microsoftml\mxLibs"

## 🚧 Enable Realtime Predictions (cmd as admin)

In [0]:
%%cmd 
cd "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\R_SERVICES\library\RevoScaleR\rxLibs\x64\"
RegisterRExt.exe /installRts
RegisterRExt.exe /installRts /database:tpcxbb_1gb
RegisterRExt.exe /installRts /database:FleitasArts

In [12]:
exec sp_configure 'show advanced options', 1;  
reconfigure;
exec sp_configure 'clr enabled', 1;  
reconfigure with override;
go
alter database tpcxbb_1gb set trustworthy on; 
exec sp_changedbowner @loginame = sa, @map = false;
alter database FleitasArts set trustworthy on;
exec sp_changedbowner @loginame = sa, @map = false;

## 🐋 Proc Uses Pre-Trained Models

In [13]:
use FleitasArts;
go
create or alter proc GetSentiment (
    @text nvarchar(max)
)
as 
    declare @script nvarchar(max);
    set @script = N'import pandas as p
from microsoftml import rx_featurize, get_sentiment
analyze_this = text
text_to_analyze = p.DataFrame(data=dict(Text=[analyze_this]))
sentiment_scores = rx_featurize(data=text_to_analyze,ml_transforms=[get_sentiment(cols=dict(scores="Text"))])
sentiment_scores["Sentiment"] = sentiment_scores.scores.apply(lambda score: "Positive" if score > 0.6 else "Negative")'
    exec sp_execute_external_script @language = N'Python',
                                    @script = @script,
                                    @output_data_1_name = N'sentiment_scores',
                                    @params = N'@text nvarchar(max)',
                                    @text = @text
    with result sets (("Text" nvarchar(max), "Score" float, "Sentiment" nvarchar(30)));
go 
exec dbo.GetSentiment N'These are not a normal stress reliever. First of all, they got sticky, hairy and dirty on the first day I received them. Second, they arrived with tiny wrinkles in their bodies and they were cold. Third, their paint started coming off. Fourth when they finally warmed up they started to stick together. Last, I thought they would be foam but, they are a sticky rubber. If these were not rubber, this review would not be so bad.';
exec dbo.GetSentiment N'These are the cutest things ever!! Super fun to play with and the best part is that it lasts for a really long time. So far these have been thrown all over the place with so many of my friends asking to borrow them because they are so fun to play with. Super soft and squishy just the perfect toy for all ages.'
exec dbo.GetSentiment N'I really did not like the taste of it' 
exec dbo.GetSentiment N'It was surprisingly quite good!'
exec dbo.GetSentiment N'I will never ever ever go to that place again!!' 
exec dbo.GetSentiment N'Destiny is a gift. Some go their entire lives, living existence as a quiet desperation. Never learning the truth that what feels as though a burden pushing down upon our shoulders, is actually, a sense of purpose that lifts us to greater heights. Never forget that fear is but the precursor to valor, that to strive and triumph in the face of fear, is what it means to be a hero. Don''t think, Master Jim. Become!';

Commands completed successfully.

Commands completed successfully.

: Msg 39023, Level 16, State 1, Procedure sp_execute_external_script, Line 1
'sp_execute_external_script' is disabled on this instance of SQL Server. Use sp_configure 'external scripts enabled' to enable it.

: Msg 11536, Level 16, State 1, Procedure dbo.GetSentiment, Line 12
EXECUTE statement failed because its WITH RESULT SETS clause specified 1 result set(s), but the statement only sent 0 result set(s) at run time.

Total execution time: 00:00:00.005

## 🐋 Table to Store MLM in Database

In [5]:
drop table if exists models;
create table models (
    ModelID         int             identity(1,1) primary key clustered,
    Language		varchar(30)     not null default('Python'),
    Name		    nvarchar(30)    not null,
    Model			varbinary(max)  ,
    CreatedOn	    datetime        default(getdate()),
    CreatedBy		nvarchar(500)   default(suser_sname())  
);
go